In [ ]:
import torch
import cv2
import os
import numpy as np
from google.colab.patches import cv2_imshow
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import math
from PIL import Image
import torch.nn as nn
import yaml
device = "cuda" if torch.cuda.is_available() else "cpu" # 檢查是否有可用的 CUDA 設備（通常是顯卡，支援 GPU 運算），如果有，就將 device 變數設置為 "cuda"，否則設置為 "cpu"。

In [ ]:
def load_config(file_path):
    with open(file_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/config.yml'
config = load_config(file_path)

In [ ]:
target_size = (config['data']['image_size'], config['data']['image_size'])

In [ ]:
def image_folder_to_tensor(folder_path):

    images_list = []

    # 获取文件夹中所有文件的列表
    # files = os.listdir(folder_path)
    files = sorted(os.listdir(folder_path))

    for file in files:

        file_path = os.path.join(folder_path, file)

        img = cv2.imread(file_path)
        img = cv2.resize(img, target_size, interpolation = cv2.INTER_LANCZOS4)
        images_list.append(img)



    img2 = np.stack(images_list, axis=0)
    tensor = torch.tensor(img2)

    tensor = tensor.to(float)


    tensor = tensor / 255.0
    tensor = tensor * 2.0
    tensor = tensor - 1.0
    tensor = tensor.permute(0, 3, 1, 2)
    # output = torch.tensor(img2)
    # output = output.permute(0, 3, 1, 2)

    # output = tensor


    return tensor



In [ ]:
def image_tensor_to_numpy(tensor):

    tensor = tensor.permute(0, 2, 3, 1)
    output = tensor.numpy()
    output = output + 1.0
    output = output / 2.0
    output = output * 255.0
    output = output.astype(np.uint8)



    return output

In [ ]:
folder_path = "/content/drive/MyDrive/Colab Notebooks/Simple_DE/image"
img = image_folder_to_tensor(folder_path)

In [ ]:
def depth_folder_to_tensor(folder_path):

    images_list = []



    files = sorted(os.listdir(folder_path))

    for file in files:

        file_path = os.path.join(folder_path, file)
        depth = np.array(Image.open(file_path), dtype=np.int16)
        assert(np.max(depth) > 255)
        depth = cv2.resize(depth, target_size, cv2.INTER_LANCZOS4)
        depth = depth.astype(np.float16) / 256.0
        images_list.append(depth)



    img2 = np.stack(images_list, axis=0)
    tensor = torch.tensor(img2)

    mini = torch.min(tensor[tensor != 0])
    tensor = (tensor - mini) / (tensor.max() - mini)
    tensor = torch.where(tensor < 0, -1, tensor)






    return tensor

In [ ]:
folder_path = "/content/drive/MyDrive/Colab Notebooks/Simple_DE/groundtruth_depth"
dep = depth_folder_to_tensor(folder_path)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img, depth):
        self.img = img
        self.depth = depth

    def __len__(self):
        return len(self.img)

    def __getitem__(self, idx):
        sample = {'img': self.img[idx], 'depth': self.depth[idx]}
        return sample

In [ ]:
custom_dataset = CustomDataset(img, dep)
batch_size = 32
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
def get_beta_schedule(beta_schedule, *, beta_start, beta_end, num_diffusion_timesteps):
    def sigmoid(x):
        return 1 / (np.exp(-x) + 1)

    if beta_schedule == "quad":
        betas = (
            np.linspace(
                beta_start ** 0.5,
                beta_end ** 0.5,
                num_diffusion_timesteps,
                dtype=np.float64,
            )
            ** 2
        )
    elif beta_schedule == "linear":
        betas = np.linspace(
            beta_start, beta_end, num_diffusion_timesteps, dtype=np.float64
        )
    elif beta_schedule == "const":
        betas = beta_end * np.ones(num_diffusion_timesteps, dtype=np.float64)
    elif beta_schedule == "jsd":  # 1/T, 1/(T-1), 1/(T-2), ..., 1
        betas = 1.0 / np.linspace(
            num_diffusion_timesteps, 1, num_diffusion_timesteps, dtype=np.float64
        )
    elif beta_schedule == "sigmoid":
        betas = np.linspace(-6, 6, num_diffusion_timesteps)
        betas = sigmoid(betas) * (beta_end - beta_start) + beta_start
    else:
        raise NotImplementedError(beta_schedule)
    assert betas.shape == (num_diffusion_timesteps,)
    return betas

In [ ]:
def compute_alpha(beta, t): # t給tensor 一維的
    beta = torch.cat([torch.zeros(1).to(beta.device), beta], dim=0)
    a = (1 - beta).cumprod(dim=0).index_select(0, t + 1).view(-1, 1, 1, 1)
    return a

In [ ]:
def get_timestep_embedding(timesteps, embedding_dim):

    assert len(timesteps.shape) == 1

    half_dim = embedding_dim // 2
    emb = math.log(10000) / (half_dim - 1)
    emb = torch.exp(torch.arange(half_dim, dtype=torch.float32) * -emb)
    emb = emb.to(device=timesteps.device)
    emb = timesteps.float()[:, None] * emb[None, :]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)
    if embedding_dim % 2 == 1:  # zero pad
        emb = torch.nn.functional.pad(emb, (0, 1, 0, 0))
    return emb

In [ ]:
def Normalize(in_channels):
    return torch.nn.GroupNorm(num_groups=32, num_channels=in_channels, eps=1e-6, affine=True)

In [ ]:
def nonlinearity(x):

    return x*torch.sigmoid(x)

In [ ]:
class Upsample(nn.Module): # this
    def __init__(self, in_channels, with_conv):
        super().__init__()
        self.with_conv = with_conv
        if self.with_conv:
            self.conv = torch.nn.Conv2d(in_channels,  # this conv let the size unchanged
                                        in_channels,
                                        kernel_size=3,
                                        stride=1,
                                        padding=1)

    def forward(self, x):
        x = torch.nn.functional.interpolate(
            x, scale_factor=2.0, mode="nearest") # double the size
        if self.with_conv:
            x = self.conv(x)
        return x

In [ ]:
class Downsample(nn.Module):
    def __init__(self, in_channels, with_conv):
        super().__init__()
        self.with_conv = with_conv
        if self.with_conv:
            # no asymmetric padding in torch conv, must do it ourselves
            self.conv = torch.nn.Conv2d(in_channels,  # halves the size
                                        in_channels,
                                        kernel_size=3,
                                        stride=2,
                                        padding=0)

    def forward(self, x):
        if self.with_conv:
            pad = (0, 1, 0, 1)
            x = torch.nn.functional.pad(x, pad, mode="constant", value=0) # 此動作相當於在每個圖片的channel的右邊下面pad 0
            x = self.conv(x)
        else:
            x = torch.nn.functional.avg_pool2d(x, kernel_size=2, stride=2)
        return x

In [ ]:
class ResnetBlock(nn.Module):
    def __init__(self, *, in_channels, out_channels=None, conv_shortcut=False,
                 dropout, temb_channels=512):
        super().__init__()
        self.in_channels = in_channels
        out_channels = in_channels if out_channels is None else out_channels
        self.out_channels = out_channels
        self.use_conv_shortcut = conv_shortcut

        self.norm1 = Normalize(in_channels)     # 這裡上面define的Normalize有點像是class的感覺
        self.conv1 = torch.nn.Conv2d(in_channels, # size unchanged
                                     out_channels,
                                     kernel_size=3,
                                     stride=1,
                                     padding=1)
        self.temb_proj = torch.nn.Linear(temb_channels,
                                         out_channels)
        self.norm2 = Normalize(out_channels)
        self.dropout = torch.nn.Dropout(dropout) # param為機率
        self.conv2 = torch.nn.Conv2d(out_channels, # size unchanged
                                     out_channels,
                                     kernel_size=3,
                                     stride=1,
                                     padding=1)
        if self.in_channels != self.out_channels:
            if self.use_conv_shortcut:
                self.conv_shortcut = torch.nn.Conv2d(in_channels,   # size unchanged
                                                     out_channels,
                                                     kernel_size=3,
                                                     stride=1,
                                                     padding=1)
            else:
                self.nin_shortcut = torch.nn.Conv2d(in_channels,    # size unchanged
                                                    out_channels,
                                                    kernel_size=1,
                                                    stride=1,
                                                    padding=0)

    def forward(self, x, temb):
        h = x
        h = self.norm1(h)    # normalize
        h = nonlinearity(h)  # sigmoid
        h = self.conv1(h)    # channel become out_channel

        h = h + self.temb_proj(nonlinearity(temb))[:, :, None, None] # 後面加入None增加空的維度，針對temb_proj(nonlinearity(temb))使用，使其可以跟h相加，但是是使用broadcasting的方式
        # temb是什麼 是用來做positional embedding的嗎????????

        h = self.norm2(h)
        h = nonlinearity(h)
        h = self.dropout(h)
        h = self.conv2(h)

        if self.in_channels != self.out_channels:  # 如果inchannel和outchannel不同需要把輸入值channel也調整成一樣，用上conv2D，若inchannel和outchannel一樣就直接加
            if self.use_conv_shortcut:
                x = self.conv_shortcut(x)
            else:
                x = self.nin_shortcut(x)

        return x+h

In [ ]:
class AttnBlock(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.in_channels = in_channels

        self.norm = Normalize(in_channels)
        self.q = torch.nn.Conv2d(in_channels, # in_cha == out_cha and the kernel size = 1, and size unchanged
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.k = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.v = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.proj_out = torch.nn.Conv2d(in_channels,
                                        in_channels,
                                        kernel_size=1,
                                        stride=1,
                                        padding=0)

    def forward(self, x):
        h_ = x
        h_ = self.norm(h_)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)

        # compute attention
        b, c, h, w = q.shape # (batch, channel, height, width)
        q = q.reshape(b, c, h*w)
        q = q.permute(0, 2, 1)   # b,hw,c.  # 此兩變換(reshape + permute)詳細情形如下格，簡單來說最外圈還是每一個data(一張照片)，
                                # 往內一圈則是把該資料的所有channel(rgb)的同個位置放在一起
        k = k.reshape(b, c, h*w)  # b,c,hw # 單一此變換則是外圈是data，向內一圈則是該data一個channel內所有的值
        w_ = torch.bmm(q, k)     # b,hw,hw    w[b,i,j]=sum_c q[b,i,c]k[b,c,j]
        # 注意這邊是q * k，是把不同channels的同個位置變成vector然後內積，這樣跟cnn最大的差別是cnn只會在同一個區塊做相關性，attention卻在channels中的每個位置會相互做相關性
        w_ = w_ * (int(c)**(-0.5)) # 為何不是 * (int(h * w) ** (-0.5))?
        w_ = torch.nn.functional.softmax(w_, dim=2)

        # attend to values
        v = v.reshape(b, c, h*w)
        w_ = w_.permute(0, 2, 1)   # b,hw,hw (first hw of k, second of q)
        # b, c,hw (hw of q) h_[b,c,j] = sum_i v[b,c,i] w_[b,i,j]
        h_ = torch.bmm(v, w_)
        h_ = h_.reshape(b, c, h, w)

        h_ = self.proj_out(h_)

        return x+h_

In [ ]:
class Model(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        ch, out_ch, ch_mult = config['model']['ch'], config['model']['out_ch'], tuple(config['model']['ch_mult'])
        num_res_blocks = config['model']['num_res_blocks']
        attn_resolutions = config['model']['attn_resolutions']
        dropout = config['model']['dropout']
        in_channels = config['model']['in_channels']
        resolution = config['data']['image_size']
        resamp_with_conv = config['model']['resamp_with_conv']
        num_timesteps = config['diffusion']['num_diffusion_timesteps']

        if config['model']['type'] == 'bayesian':
            self.logvar = nn.Parameter(torch.zeros(num_timesteps))

        self.ch = ch
        self.temb_ch = self.ch*4
        self.num_resolutions = len(ch_mult)
        self.num_res_blocks = num_res_blocks
        self.resolution = resolution
        self.in_channels = in_channels

        # timestep embedding
        self.temb = nn.Module()
        self.temb.dense = nn.ModuleList([
            torch.nn.Linear(self.ch,
                            self.temb_ch),
            torch.nn.Linear(self.temb_ch,
                            self.temb_ch),
        ])

        # downsampling
        self.conv_in = torch.nn.Conv2d(in_channels,
                                       self.ch,
                                       kernel_size=3,
                                       stride=1,
                                       padding=1)

        curr_res = resolution
        in_ch_mult = (1,)+ch_mult
        self.down = nn.ModuleList()
        block_in = None
        for i_level in range(self.num_resolutions):
            block = nn.ModuleList()
            attn = nn.ModuleList()
            block_in = ch*in_ch_mult[i_level]
            block_out = ch*ch_mult[i_level]
            for i_block in range(self.num_res_blocks):
                block.append(ResnetBlock(in_channels=block_in,
                                         out_channels=block_out,
                                         temb_channels=self.temb_ch,
                                         dropout=dropout))
                block_in = block_out
                if curr_res in attn_resolutions:
                    attn.append(AttnBlock(block_in))
            down = nn.Module()
            down.block = block
            down.attn = attn
            if i_level != self.num_resolutions-1:
                down.downsample = Downsample(block_in, resamp_with_conv)
                curr_res = curr_res // 2
            self.down.append(down)

        # middle
        self.mid = nn.Module()
        self.mid.block_1 = ResnetBlock(in_channels=block_in,
                                       out_channels=block_in,
                                       temb_channels=self.temb_ch,
                                       dropout=dropout)
        self.mid.attn_1 = AttnBlock(block_in)
        self.mid.block_2 = ResnetBlock(in_channels=block_in,
                                       out_channels=block_in,
                                       temb_channels=self.temb_ch,
                                       dropout=dropout)

        # upsampling
        self.up = nn.ModuleList()
        for i_level in reversed(range(self.num_resolutions)):
            block = nn.ModuleList()
            attn = nn.ModuleList()
            block_out = ch*ch_mult[i_level]
            skip_in = ch*ch_mult[i_level]
            for i_block in range(self.num_res_blocks+1):
                if i_block == self.num_res_blocks:
                    skip_in = ch*in_ch_mult[i_level] # 最後一個block時inchannel數可能變少
                block.append(ResnetBlock(in_channels=block_in+skip_in,
                                         out_channels=block_out,
                                         temb_channels=self.temb_ch,
                                         dropout=dropout))
                block_in = block_out
                if curr_res in attn_resolutions:
                    attn.append(AttnBlock(block_in))
            up = nn.Module()
            up.block = block
            up.attn = attn
            if i_level != 0:
                up.upsample = Upsample(block_in, resamp_with_conv)
                curr_res = curr_res * 2
            self.up.insert(0, up)  # prepend to get consistent order, (0, up)代表在ModuleList()的0位置插入up這個Module

        # end
        self.norm_out = Normalize(block_in)
        self.conv_out = torch.nn.Conv2d(block_in,
                                        out_ch,
                                        kernel_size=3,
                                        stride=1,
                                        padding=1)

    def forward(self, x, t):
        assert x.shape[2] == x.shape[3] == self.resolution # to check if the height and width are the same with the resolution

        # timestep embedding
        temb = get_timestep_embedding(t, self.ch)
        temb = self.temb.dense[0](temb)
        temb = nonlinearity(temb)
        temb = self.temb.dense[1](temb)

        # downsampling down 裡面有好幾個元素，每個元素包含block(resblock), attn, downsample，其中attn只有幾個元素會有，downsample除了最後一個元素以外都有
        # 整個流程就是把x送進conv2D 然後送進down裡面經過resblock和部份attn downsample(conv2D)
        hs = [self.conv_in(x)]
        for i_level in range(self.num_resolutions):
            for i_block in range(self.num_res_blocks):
                h = self.down[i_level].block[i_block](hs[-1], temb) # h 如果可以是attn就是attn 不然就是res, note that h是把值喂進去模塊後的值，要把hs的最後跟time embedded送進去
                if len(self.down[i_level].attn) > 0:
                    h = self.down[i_level].attn[i_block](h)
                hs.append(h)
            if i_level != self.num_resolutions-1:
                hs.append(self.down[i_level].downsample(hs[-1]))

        # middle
        h = hs[-1]
        h = self.mid.block_1(h, temb)
        h = self.mid.attn_1(h)
        h = self.mid.block_2(h, temb)

        # upsampling
        for i_level in reversed(range(self.num_resolutions)):
            for i_block in range(self.num_res_blocks+1):
                h = self.up[i_level].block[i_block](torch.cat([h, hs.pop()], dim=1), temb) # u-net的cat down
                if len(self.up[i_level].attn) > 0:
                    h = self.up[i_level].attn[i_block](h)
            if i_level != 0:
                h = self.up[i_level].upsample(h)

        # end
        h = self.norm_out(h)
        h = nonlinearity(h)
        h = self.conv_out(h)
        return h